In [ ]:
import os
import cv2
import dlib
import numpy as np
import argparse
import inception_resnet_v1
import tensorflow as tf
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
from imutils import face_utils

In [ ]:
def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
              font_scale=1, thickness=2):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y, size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness=2)

In [ ]:
if __name__ == '__main__':
    model_path = "../gender_age_tf/models/"
    
    sess = tf.Session()
    images_pl = tf.placeholder(tf.float32, shape=[None, 160, 160, 3], name='input_image')
    images_norm = tf.map_fn(lambda frame:
                           tf.image.per_image_standardization(frame), images_pl)
    train_mode = tf.placeholder(tf.bool)
    age_logits, gender_logits, _ = inception_resnet_v1.inference(images_norm, keep_probability=0.8, phase_train=train_mode, weight_decay=1e-5)
    
    gender = tf.argmax(tf.nn.softmax(gender_logits), 1)
    age_ = tf.cast(tf.constant([i for i in range(0, 101)]), tf.float32)
    age = tf.reduce_sum(tf.multiply(tf.nn.softmax(age_logits), age_), axis=1)
    init_op = tf.group(tf.global_variables_initializer(), 
                      tf.local_variables_initializer())
    
    sess.run(init_op)
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(model_path)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print("restore model!")
    else:
        print("don't find model!")
    
    
    # Depth of network
    depth = 16
    
    # width of network
    width = 8
    
    # For face detection
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    fa_image = FaceAligner(predictor, desiredFaceWidth=160)
    
    img_size = 160
    
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    while True:
        # get video frame
        ret, img = cap.read()
        
#         if not ret:
#             print("error: failed to capture image")
#             return -1
        
        input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # height, width of image
        img_h, img_w, _ = np.shape(input_img)
        
        # detect faces using dlib detector
        detected = detector(gray, 1)
        faces = np.empty((len(detected), img_size, img_size, 3))
        #
        for i, d in enumerate(detected): 
            # get axis position of face
#             shape = predictor(gray, d)
#             shape = face_utils.shape_to_np(shape)
            
#             # Compate eye right and eye left
#             center_face = shape[37][1] - shape[46][1]
#             print(center_face)
#             if center_face < 10 & center_face > -10:
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            faces[i, :, :, :] = fa_image.align(input_img, gray, detected[i])
    # Detect face when the face is center
        
        if len(detected) > 0:
            ages, genders = sess.run([age, gender], feed_dict={images_pl: faces})
        
        # draw results
        for i, d in enumerate(detected):
            label = "{}, {}".format(int(ages[i]), "F" if genders[i] == 0 else "M")
            draw_label(img, (d.left(), d.top()), label)

        cv2.imshow("result", img)
        key = cv2.waitKey(1)

        if key == 27:
            break

In [ ]:
import os
import cv2
import dlib
import numpy as np
import argparse
import inception_resnet_v1
import tensorflow as tf
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
from imutils import face_utils

In [ ]:
def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=1, thickness=2):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness)


In [3]:
def main(sess,age,gender,train_mode,images_pl):
#     args = get_args()
    depth = 16
    k = 8

    # for face detection
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    fa = FaceAligner(predictor, desiredFaceWidth=160)

    # load model and weights
    img_size = 160

    # capture video
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    
    num_frame = 0
    
    while True:
        # get video frame
        ret, img = cap.read()
            
        if not ret:
            print("error: failed to capture image")
            return -1
        num_frame += 1
        
        if num_frame % 3 == 0:
            input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img_h, img_w, _ = np.shape(input_img)

            # detect faces using dlib detector
            detected = detector(input_img, 1)
            faces = np.empty((len(detected), img_size, img_size, 3))

            for i, d in enumerate(detected):

                shape = predictor(gray, d)
                shape = face_utils.shape_to_np(shape)

                center_face = shape[37][1] - shape[46][1]
                print(center_face)
                if center_face < 20 and center_face > -20:
                    x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
        #             xw1 = max(int(x1 - 0.4 * w), 0)
        #             yw1 = max(int(y1 - 0.4 * h), 0)
        #             xw2 = min(int(x2 + 0.4 * w), img_w - 1)
        #             yw2 = min(int(y2 + 0.4 * h), img_h - 1)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    # cv2.rectangle(img, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
                    faces[i, :, :, :] = fa.align(input_img, gray, detected[i])
                    # faces[i,:,:,:] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))
            #
                    if len(detected) > 0:
                        # predict ages and genders of the detected faces
                        ages,genders = sess.run([age, gender], feed_dict={images_pl: faces, train_mode: False})

                    # draw results
                    for i, d in enumerate(detected):
                        label = "{}, {}".format(int(ages[i]), "F" if genders[i] == 0 else "M")
                        draw_label(img, (d.left(), d.top()), label)

            cv2.imshow("result", img)
        key = cv2.waitKey(1)

        if key == 27:
            break

In [4]:
def load_network(model_path):
    sess = tf.Session()
    images_pl = tf.placeholder(tf.float32, shape=[None, 160, 160, 3], name='input_image')
    images_norm = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), images_pl)
    train_mode = tf.placeholder(tf.bool)
    age_logits, gender_logits, _ = inception_resnet_v1.inference(images_norm, keep_probability=0.8,
                                                                 phase_train=train_mode,
                                                                 weight_decay=1e-5)
    gender = tf.argmax(tf.nn.softmax(gender_logits), 1)
    age_ = tf.cast(tf.constant([i for i in range(0, 101)]), tf.float32)
    age = tf.reduce_sum(tf.multiply(tf.nn.softmax(age_logits), age_), axis=1)
    init_op = tf.group(tf.global_variables_initializer(),
                       tf.local_variables_initializer())
    sess.run(init_op)
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(model_path)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print("restore model!")
    else:
        pass
    return sess,age,gender,train_mode,images_pl


In [ ]:
if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--model_path", "--M", default="./models", type=str, help="Model Path")
#     args = parser.parse_args()
    sess, age, gender, train_mode,images_pl = load_network("../gender_age_tf/models/")
    main(sess,age,gender,train_mode,images_pl)

INFO:tensorflow:Restoring parameters from ../gender_age_tf/models/model.ckpt-14001
restore model!
-9
-10
-8
-9
-9
-9
-7
-7
-6
-7
-7
-7
-6
-4
-7
-8
-7
-7
-7
-3
-7
-7
-8
-8
-6
-7
-8
-7
-7
-7
-7
-7
-7
-6
-6
-14
-24
-24
-23
-22
-24
-23
-18
-22
-18
-21
-22
-21
-21
-21
-21
-30
-42
-41
-38
-38
-36
-35
-36
-34
-30
-6
14
17
19
20
19
19
24
18
19
18
17
17
18
23
18
17
19
18
8
1
-5
-8
-10
-10
-12
-14
-18
-18
-16
-17
-17
-17
-17
-17
-12
-10
-8
-7
-7
-7
-8
-8
-7
-7
-7
-8
-8
-10
-10
-8
-5
-5
-7
-7
-8
-7
-8
-8
-9
-8
-6
-3
-4
-4
-5
-4
-3
-4
-4
-4
-3
-4
-4
-1
-2
-1
-5
-8
-6
-6
-5
-4
-5
-7
-7
-5
-5
-5
-5
-2
-5
-5
-6
-7
-7
-6
-5
-5
-5
-3
-3
-4
-4
-4
-4
-4
-3
-4
-5
-3
-5
-2
-4
-3
-5
-4
-4
-4
-4
-3
-3
-3
-3
-4
-6
-8
0
-10
-4
-13
-7
-5
-12
-11
-15
-16
-11
-13
-8
-14
-13
-15
-18
-15
-22
-19
-20
-5
-7
-10
-10
-7
-7
-8
-6
-5
1
-7
-7
-10
-9
-8
-9
-6
-6
-6
-5
-4
-2
-5
-5
-5
-6
3
2
2
-3
-7
-3
-10
-4
-5
-6
-6
-4
-5
-7
-3
-3
-5
-6
-3
-8
-11
-13
-11
-12
-16
-15
-11
-11
-9
-9
-7
-3
0
-11
-9
-6
-7
-7
-7
-5
-7
-5
-12
-5


In [ ]:
sess = tf.Session()
print(sess.run(tf.cast(tf.constant([i for i in range(0, 101)]), tf.float32)))